In [ ]:
# !pip install tensorflow-determinism

!pip install tensorflow-addons==0.19.0
import tensorflow_addons as tfa

# !pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
import datetime
import pandas as pd
import os
seed_value= 42 
####*IMPORANT*: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ["TF_DETERMINISTIC_OPS"] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2, ResNet50, InceptionV3, DenseNet121
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import random
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import argparse
from PIL import Image

parser = argparse.ArgumentParser()
parser.add_argument('--gpu_node', type=str, help='specify gpu nodes', default='0')
parser.add_argument('--database', type=str, help='choose RadImageNet or ImageNet', default='RadImageNet')
parser.add_argument('--model_name', type=str, help='choose IRV2/ResNet50/DenseNet121/InceptionV3', default='IRV2')
parser.add_argument('--batch_size', type=int, help='batch size', default=32)
parser.add_argument('--image_size', type=int, help='image size', default=360)
# parser.add_argument('--epoch', type=int, help='number of epochs', default=1)
parser.add_argument('--epoch', type=int, help='number of epochs', default=50)
parser.add_argument('--structure', type=str, help='unfreezeall/freezeall/unfreezetop10', default='unfreezeall')
# parser.add_argument('--learning_rate', type=float, help='learning rate', default=0.0001)
parser.add_argument('--learning_rate', type=float, help='learning rate', default=0.001)
parser.add_argument("--fold",  type=int, help='fold number', default=9)
parser.add_argument("-f", "--file", required=False)


args = parser.parse_args()

In [ ]:
def reset_random_seeds():
  # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
  os.environ['PYTHONHASHSEED']=str(seed_value)
  os.environ["TF_DETERMINISTIC_OPS"] = '1'

  # 2. Set `python` built-in pseudo-random generator at a fixed value
  random.seed(seed_value)

  # 3. Set `numpy` pseudo-random generator at a fixed value
  np.random.seed(seed_value)

  # 4. Set `tensorflow` pseudo-random generator at a fixed value
  tf.random.set_seed(seed_value)
  tf.keras.utils.set_random_seed(seed_value)

def set_seeds():
    tf.config.experimental.enable_op_determinism()
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    os.environ["TF_DETERMINISTIC_OPS"] = '1'
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
    tf.keras.utils.set_random_seed(seed_value)
    np.random.seed(seed_value)
    

def set_global_determinism():
    set_seeds()

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

set_global_determinism()
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

In [ ]:
print(args.database)

RadImageNet


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Configuration.
path = os.getcwd()
path = path + "/gdrive/MyDrive/puc/tcc"

print(path)

/content/gdrive/MyDrive/puc/tcc


In [ ]:
# os.chdir("/gdrive")
!ls

gdrive	sample_data


In [ ]:
# Configuration.
# path = os.getcwd()

# D:\Meu Drive\puc\tcc\RadImageNet_models

train_img_folder = path+'/Datasets/360x360-dataset-category-4-split/split-augmentedV2/train' #4
val_img_folder = path+'/Datasets/360x360-dataset-category-4-split/split-augmentedV2/val' #4
print(train_img_folder)

/content/gdrive/MyDrive/puc/tcc/Datasets/360x360-dataset-category-4-split/split-augmentedV2/train


In [ ]:
# destroy the current TensorFlow graph and create a new one
backend.clear_session()
set_global_determinism()

current_time = datetime.datetime.now()
current_time_str = str(current_time).replace(":", ".")

### Limit to the first GPU for this model
os.environ["CUDA_VISIBLE_DEVICES"]= args.gpu_node

### Import pre-trained weights from ImageNet or RadImageNet
database = args.database
if not database in ['RadImageNet', 'ImageNet']:
    raise Exception('Pre-trained database not exists. Please choose ImageNet or RadImageNet')
    
if not args.structure in ['unfreezeall', 'freezeall','unfreezetop10']:
    raise Exception('Freeze any layers? Choose to unfreezeall/freezeall/unfreezetop10 layers for the network.')

### Set up training image size, batch size and number of epochs
image_size = args.image_size
batch_size = args.batch_size
num_epoches = args.epoch
fold = args.fold

### Creat model
def get_compiled_model():
    if not args.model_name in ['IRV2', 'ResNet50', 'DenseNet121', 'InceptionV3']:
        raise Exception('Pre-trained network not exists. Please choose IRV2/ResNet50/DenseNet121/InceptionV3 instead')
    else:
        if args.model_name == 'IRV2':
            if database == 'RadImageNet':
                model_dir = path + "/RadImageNet_models/RadImageNet-IRV2_notop.h5"
                base_model = InceptionResNetV2(weights=model_dir, input_shape=(image_size, image_size, 3), include_top=False,pooling='avg')
            else:
                base_model = InceptionResNetV2(weights='imagenet', input_shape=(image_size, image_size, 3),include_top=False,pooling='avg')
        if args.model_name == 'ResNet50':
            if database == 'RadImageNet':
                model_dir = path + "/RadImageNet_models/RadImageNet-ResNet50_notop.h5"
                base_model = ResNet50(weights=model_dir, input_shape=(image_size, image_size, 3), include_top=False,pooling='avg')
            else:
                base_model = ResNet50(weights='imagenet', input_shape=(image_size, image_size, 3), include_top=False,pooling='avg')
        if args.model_name == 'DenseNet121':
            if database == 'RadImageNet':
                model_dir = path + "/RadImageNet_models/RadImageNet-DenseNet121_notop.h5"
                base_model = DenseNet121(weights=model_dir, input_shape=(image_size, image_size, 3), include_top=False,pooling='avg')
            else:
                base_model = DenseNet121(weights='imagenet', input_shape=(image_size, image_size, 3), include_top=False,pooling='avg')
        if args.model_name == 'InceptionV3':
            if database == 'RadImageNet':
                model_dir = path + "/RadImageNet_models/RadImageNet-InceptionV3_notop.h5"
                base_model = InceptionV3(weights=model_dir, input_shape=(image_size, image_size, 3), include_top=False,pooling='avg') 
            else:
                base_model = InceptionV3(weights='imagenet', input_shape=(image_size, image_size, 3), include_top=False,pooling='avg')
    if args.structure == 'freezeall':
        for layer in base_model.layers:
            layer.trainable = False
    if args.structure == 'unfreezeall':
        pass
    if args.structure == 'unfreezetop10':
        for layer in base_model.layers[:-10]:
            layer.trainable = False
    y = base_model.output

    # creating a dropout layer with a 50% chance of setting inputs to zero
    y = Dropout(0.5, seed=seed_value)(y)

    predictions = Dense(num_classes, activation='softmax')(y)

    model = Model(inputs=base_model.input, outputs=predictions)
    adam = Adam(learning_rate=args.learning_rate)
    model.compile(optimizer=adam, loss=CategoricalCrossentropy(), metrics=['accuracy',
                                                                      tf.keras.metrics.Precision(name='precision'), 
                                                                      tf.keras.metrics.Recall(name='recall'),
                                                                      tfa.metrics.F1Score(num_classes=num_classes,name='f1_score', average='macro')])
    return model
    
def run_model():
    # reset_random_seeds()
    # Call the function with seed value
    set_global_determinism()

    ### Set train steps and validation steps
    model = get_compiled_model()
    train_steps =  len(train_generator.labels)/ batch_size
    val_steps = len(validation_generator.labels) / batch_size
    
    #### set the path to save models having lowest validation loss during training
    save_model_dir = path + '/models/'
    if not os.path.exists(save_model_dir):
        os.mkdir(save_model_dir) 
    filepath= path + "/models/"+ current_time_str +"-v2-binary-thyroid-sigmoid-" + args.structure + "-" + database + "-" + args.model_name + "-" + str(image_size) + "-" + str(batch_size) + "-"+str(args.learning_rate)+"-"+str(num_epoches)+ ".h5"   
  

    earlyStop = EarlyStopping(monitor='val_f1_score', mode='max', verbose=1, patience=60)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001, verbose=1)
    checkpoint = ModelCheckpoint(filepath, monitor='val_f1_score', verbose=1, save_best_only=True, mode='max')


    set_global_determinism()
    history = model.fit(
            train_generator,
            epochs=num_epoches,
            steps_per_epoch=train_steps,
            validation_data=validation_generator,
            validation_steps=val_steps,
            use_multiprocessing=False,
            callbacks=[checkpoint, earlyStop])
    
    ### Save training loss
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    train_precision = history.history['precision']
    val_precision = history.history['val_precision']
    train_recall = history.history['recall']
    val_recall = history.history['val_recall']
    train_f1_score = history.history['f1_score']
    val_f1_score = history.history['val_f1_score']
    train_loss = history.history['loss'] 
    val_loss = history.history['val_loss']
    d_loss = pd.DataFrame({'train_accuracy':train_accuracy, 'val_accuracy':val_accuracy, 'train_loss':train_loss, 'val_loss':val_loss,
                           'train_precision':train_precision, 'val_precision':val_precision, 'train_recall':train_recall, 'val_recall':val_recall,
                           'train_f1_score':train_f1_score, 'val_f1_score':val_f1_score})

    save_loss_dir = path + '/loss'
    if not os.path.exists(save_loss_dir):
        os.mkdir(save_loss_dir)
    d_loss.to_csv(path + "/loss/"+ current_time_str +"-v2-binary-thyroid-sigmoid-" + 
                  args.structure + "-" + database + "-" +  args.model_name + "-" +
                  str(image_size) + "-" + str(batch_size) + "-"+str(args.learning_rate)+"-"+
                  str(num_epoches)+ ".csv", index=False)  
    

df_train=pd.read_csv(path + "/csv/train-fold-"+str(fold)+".csv")
df_val=pd.read_csv(path + "/csv/test-fold-"+str(fold)+".csv")

data_generator = ImageDataGenerator(
    rescale = 1/255.)

train_generator = data_generator.flow_from_dataframe(
    dataframe=df_train,
    x_col = 'img_dir',
    y_col = 'label',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=True,
    seed=seed_value)

validation_generator = data_generator.flow_from_dataframe(
    dataframe=df_val,
    x_col = 'img_dir',
    y_col = 'label',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=True,
    seed=seed_value)


num_classes =len(train_generator.class_indices)
run_model()

Found 4800 validated image filenames belonging to 3 classes.
Found 560 validated image filenames belonging to 3 classes.
Epoch 1/50
150/150 [==============================] - ETA: 0s - loss: 0.9943 - accuracy: 0.4769 - precision: 0.6330 - recall: 0.2131 - f1_score: 0.4559
Epoch 1: val_f1_score improved from -inf to 0.39381, saving model to /content/gdrive/MyDrive/puc/tcc/models/2023-06-08 17.22.01.243054-v2-binary-thyroid-sigmoid-unfreezeall-RadImageNet-IRV2-360-32-0.001-50.h5
150/150 [==============================] - 407s 2s/step - loss: 0.9943 - accuracy: 0.4769 - precision: 0.6330 - recall: 0.2131 - f1_score: 0.4559 - val_loss: 1.1834 - val_accuracy: 0.3964 - val_precision: 0.4294 - val_recall: 0.2500 - val_f1_score: 0.3938
Epoch 2/50
150/150 [==============================] - ETA: 0s - loss: 0.7400 - accuracy: 0.6596 - precision: 0.7228 - recall: 0.5633 - f1_score: 0.6525
Epoch 2: val_f1_score improved from 0.39381 to 0.48087, saving model to /content/gdrive/MyDrive/puc/tcc/models